# Import Libraries 

In [ ]:
import numpy as np 

# data in/out & eda
import pandas as pd 
import pandas_profiling

# visualizations
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load Dataset

In [ ]:
# read in Dayton's power consumption data
df = pd.read_csv('/home/jovyan/Capstone_Data_Files/Capstone_Dayton_Enrgy_Data/DAYTON_hourly.csv')

# sort by date & time
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by=['Datetime'], axis=0, ascending=True, inplace=True)
df.reset_index(inplace=True, drop=True)

# renaming the target variable columns
df.rename(columns={'DAYTON_MW':'EnergyDemand_in_MW'}, inplace=True)

# display the first couple of rows
df.head()

# Cleaning the dataset

Deduplicating

In [ ]:
# deduplicate to remove redundancies, only keeping the last measurement per datetime
df.drop_duplicates(subset='Datetime', keep='last', inplace=True)

Find and Fill Missing DateTime Instances

In [ ]:
# Checking if we have a continuous dataset
df = df.set_index('Datetime')
print(f'df.index.freq is set to: {df.index.freq}')

Here, datetime index's frequency set to None is an indication that there are some missing data points somewhere (otherwise Python could deduce it). Let's compare it to an uninterruped custom date range.

In [ ]:
# setting up a custom range
date_range = pd.date_range(start=min(df.index), 
                           end=max(df.index), 
                           freq='H')

In [ ]:
print(f'The difference in length between the custom date range and our dataset is {(len(date_range)-len(df))}:')
print(date_range.difference(df.index))

Reindexing our dataset, then performing imputation

In [ ]:
# this will append the previously missing datetimes, and create null values in our target variable
df = df.reindex(date_range)

# we fill in the blanks with values that lie on a linear curve between existing data points
df['EnergyDemand_in_MW'].interpolate(method='linear', inplace=True)

# now we have a neatly continuous datetime index
print(f'The df.index.freq is now: {df.index.freq}, indicating that we no longer have missing instances')

# Extracting Time Features

Splitting up the date-timestamp column into its different components will allow us to find patterns for different groups.

In [ ]:
df['dow'] = df.index.dayofweek
df['doy'] = df.index.dayofyear
df['year'] = df.index.year
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['hour'] = df.index.hour
df['weekday'] = df.index.weekday_name
df['woy'] = df.index.weekofyear
df['dom'] = df.index.day # Day of Month
df['date'] = df.index.date 

# Adding the season number
df['season'] = df['month'].apply(lambda month_number: (month_number%12 + 3)//3)

# Exploratory Data Analysis (EDA)

pandas_profiling to get an overview of our dataset 

In [ ]:
pandas_profiling.ProfileReport(df)

Correlation matrix indicates that with strongest correlations among all other variables, "dow" (day of week) and "hour" will be interesting to look at in the context of predicting our target variable.

# Quick Visuals 

Plotting the energy consumption over time

In [ ]:
# Since plotly doesn't allow us to access the index, let's copy it into a column 
df['date_and_time'] = df.index

# plotting
fig = px.line(df,
              x='date_and_time',
              y='EnergyDemand_in_MW',
              title=f'Power Demand (MW) over time [{min(df.year)} - {max(df.year)}]')
fig.update_traces(line=dict(width=0.05))
fig.update_layout(xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

We can definitely identify a seasonal pattern here, but there does not seem to be any immediately apparent trend.

Date and Time Patterns

Let's use our previously extracted date and time features to see if recurring patterns emerge from the aggregated data. Take for instance, the power demand throughout the day for each weekday:

In [ ]:
# aggregated data
_ = df\
    .groupby(['hour', 'weekday'], as_index=False)\
    .agg({'EnergyDemand_in_MW':'median'})

# plotting
fig = px.line(_, 
              x='hour', 
              y='EnergyDemand_in_MW', 
              color='weekday', 
              title='Median Hourly Power Demand per Weekday')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='Energy Demand [MW]')
fig.show()

It is clear that the deamand for electricity is lower during the weekends, and dips a little sooner on Friday afternoons.

Now, let's look at power demand per season:

In [ ]:
# aggregated data
_ = df\
    .groupby(['hour', 'season'], as_index=False)\
    .agg({'EnergyDemand_in_MW':'median'})

# plotting
fig = px.line(_,
              x='hour', 
              y='EnergyDemand_in_MW', 
              color='season', 
              title='Median Hourly Power Demand per Season')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='Energy Demand [MW]')
fig.show()

Clearly, those air conditioners are turned up during the summer afternoons!

# Decomposing the Time-Series

Data points over time can have both a trend (upward or downward) and/or seasonality. As we have established in our EDA, these aspects seem to play a role in this dataset.

Because the seasonal variation in our dataset appears constant over time as indicated by the repeating spikes with about the same level of increase and decrease in the "Power Demand Over Time" chart, we will use the additive model for decomposition (as opposed to the multiplicative model, which is useful for cases where seasonal variation increases over time).

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# seasonal_decompose needs a dataframe with a datetime index
series = df[['EnergyDemand_in_MW']]
frequency = 24*365

# decomposing the time-series, with the frequency being 24 hours per 365 days
decomposed = seasonal_decompose(series, model='additive', freq=frequency)

In [ ]:
# plotting the different elements constituting our time-series
def plot_decompositions(decompositions, titles, line_widths):
    for d, t, lw in zip(decompositions, titles, line_widths):
        
        # draw a line plot of the data
        fig = px.line(d,
              y='EnergyDemand_in_MW',
              title=t,
              height=300)
        
        # adjust line width
        fig.update_traces(line=dict(width=lw))
        
        # change layout of axes and the figure's margins 
        # to emulate tight_layout
        fig.update_layout(
            xaxis=dict(
                showticklabels=False,
                linewidth=1
            ),
            yaxis=dict(title=''),
            margin=go.layout.Margin(
                l=40, r=40, b=0, t=40, pad=0
            ),
        )
        
        # display
        fig.show()

# calling the function 
plot_decompositions(decompositions=[decomposed.trend, 
                                    decomposed.seasonal, 
                                    decomposed.resid],
                    titles=['Trend', 
                            'Seasonality',
                            'Residuals'],
                    line_widths=[2, 0.025, 0.05])

# Forecasting Models

We will look at the following methods:

Triple Exponential Smoothing: Holt-Winter's
Explicit Multi-Seasonality: Prophet

# Train/Test
The goal is to accurately predict up to 12 month's worth of energy demand. We will restrict our training data to a couple of years leading up to that, to make sure we don't capture any outdated trends (industry shifts). This doesn't seem to be the case, judging from our prior visualisations, but it will keep our computational load low and allow us to iterate over models more quickly.

Let's first find out last date for which the data are available, so we can eventually compare the forecast with the ground truth.

In [ ]:
f'The last date time point in our dataframe is: {max(df.index)}'

In [ ]:
# manually setting the cutoff date
CUTOFF_DATE = pd.to_datetime('2017-08-01')
TIME_DELTA = pd.DateOffset(years=8)

# splitting in training and testing datasets
train = df.loc[(df.index < CUTOFF_DATE) & (df.index >= CUTOFF_DATE-TIME_DELTA) ].copy()
test = df.loc[df.index >= CUTOFF_DATE].copy()

In [ ]:
print(f'Training shape: {train.shape} \nTesting shape: {test.shape}\n')
print(f'The training set lies between the dates: {min(train.index)} and {max(train.index)}')
print(f'For the testing set, the dates are: {min(test.index)} and {max(test.index)}')

In [ ]:
# Saving the train and test datasets
train.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/train.csv', index = False)
test.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/test.csv', index = False)

# Holt-Winter 

In [ ]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
import statsmodels.api as sm

# exponential smoothing only takes into consideration patterns in the target variable
# so we discard the other features
exp_smooth_train, exp_smooth_test = train['EnergyDemand_in_MW'], test['EnergyDemand_in_MW']

# fit & predict
holt_winter = sm.tsa.ExponentialSmoothing(exp_smooth_train,
                                          seasonal_periods=24*365,
                                          seasonal='add').fit()
y_hat_holt_winter = holt_winter.forecast(len(exp_smooth_test))

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=exp_smooth_test.index, y=exp_smooth_test,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=y_hat_holt_winter.index, y=y_hat_holt_winter,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Holt-Winter Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')

Not bad for an algorithm that only considers patterns in the history of the target variable!

We can see it fall short around the winter holiday period until March and late during the summer, but it has clearly recognised the frequency and the degree of variance of the seasonal patterns. Let's quantify its performance.

In [ ]:
def mape(y_true, y_pred):
    """ Mean Absolute Percentage Error """
    
    # convert to numpy arrays
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # take the percentage error
    pe = (y_true - y_pred) / y_true
    
    # take the absolute values
    ape = np.abs(pe)
    
    # quantify the performance in a single number
    mape = np.mean(ape)
    
    return f'{mape*100:.2f}%'

In [ ]:
mape_hw = mape(y_true=exp_smooth_test, y_pred=y_hat_holt_winter)
print(f'Our Holt-Winter model has a mean average percentage error of {mape_hw}')

Now, let's look at the Holt-Winter model in greater detail 
Let's look at some intra-day predictions at the beginning and the end of the requested forecast.

In [ ]:
# interval length set to 92 days (approximately a fiscal quarter)
interval = 24 * 92

# intermediary variables for readability
x_true, y_true = exp_smooth_test.iloc[:interval].index, exp_smooth_test.iloc[:interval]
x_pred, y_pred = y_hat_holt_winter.iloc[:interval].index, y_hat_holt_winter.iloc[:interval]

# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Holt-Winter Intra-Day Forecast of First {interval} Hours of Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for interval of the first {interval} hours: {mape(y_true, y_pred)}')

In [ ]:
# interval length set to 92 days (approximately a fiscal quarter)
interval = -24 * 92

# intermediary variables for readability
x_true, y_true = exp_smooth_test.iloc[interval:].index, exp_smooth_test.iloc[interval:]
x_pred, y_pred = y_hat_holt_winter.iloc[interval:].index, y_hat_holt_winter.iloc[interval:]

# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Holt-Winter Intra-Day Forecast of Last {abs(interval)} Hours of Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for interval of the last {abs(interval)} hours: {mape(y_true, y_pred)}')

# The Issue of Double/Triple Seasonality
The Holt-Winters method crams all the seasonalities it can find in a single variable. From the decomposition, we can also tell it mistakes some seasonality for trend. While it seems to be a decent predictor of the overal trends, its precision could be fine tuned. From the model, however, it is not immediately obvious how to do that.

Other methods, like SARIMA (seasonal ARIMA) allow you to specify the interval of different lags at which seasonality occurs more precisely. You can expose these by looking at autocorrelation, and configure your model accordingly.

Auto-Correlogram & Partial Auto-Correlogram
Below plots distinctly expose the daily recurring element (each lag is an hour, with spikes at the 24, 48, and 72 marks). Intuitively, this should make sense: If you want to know how much energy is going to be used tomorrow, changes are high it will be highly correlated to today's levels barring external variables like weather. 

In [ ]:
#we use tra.diff()(differenced data), because this time series is unit root process.
fig,ax = plt.subplots(2,1,figsize=(20,10))
fig = sm.graphics.tsa.plot_acf( train['EnergyDemand_in_MW'].diff().dropna(), lags=72, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(train['EnergyDemand_in_MW'].diff().dropna(), lags=72, ax=ax[1])
plt.show()

Of course, we have already exposed the weekly and yearly seasonal element during our exploratory data analysis. Checking for autocorrelation at lags 168 (24 x 7, weekly) and 8760 (24 x 365, yearly), respectively, would require a lot of memory. Instead, let's look at a model explicitly designed to account for multiple seasonalities.

In [ ]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation

In [ ]:
# format data for prophet model using 'ds' and 'y'
train_prophet = train[['EnergyDemand_in_MW']]\
                    .reset_index()\
                    .rename(columns={
                        'index':'ds', 
                        'EnergyDemand_in_MW':'y'
                    })

test_prophet = test[['EnergyDemand_in_MW']]\
                    .reset_index()\
                    .rename(columns={
                        'index':'ds',
                        'EnergyDemand_in_MW':'y'
                    })

In [ ]:
# conditions
def is_spring(ds):
    date = pd.to_datetime(ds)
    return (date.month >= 3) & (date.month <= 5)

def is_summer(ds):
    date = pd.to_datetime(ds)
    return (date.month >= 6) & (date.month <= 8)

def is_autumn(ds):
    date = pd.to_datetime(ds)
    return (date.month >= 9) & (date.month <= 11)

def is_winter(ds):
    date = pd.to_datetime(ds)
    return (date.month >= 12) | (date.month <= 2)

def is_weekend(ds):
    date = pd.to_datetime(ds)
    return date.weekday_name in ('Saturday', 'Sunday')

# adding to train set
train_prophet['is_spring'] = train_prophet['ds'].apply(is_spring)
train_prophet['is_summer'] = train_prophet['ds'].apply(is_summer)
train_prophet['is_autumn'] = train_prophet['ds'].apply(is_autumn)
train_prophet['is_winter'] = train_prophet['ds'].apply(is_winter)
train_prophet['is_weekend'] = train_prophet['ds'].apply(is_weekend)
train_prophet['is_weekday'] = ~train_prophet['ds'].apply(is_weekend)

# adding to test set
test_prophet['is_spring'] = test_prophet['ds'].apply(is_spring)
test_prophet['is_summer'] = test_prophet['ds'].apply(is_summer)
test_prophet['is_autumn'] = test_prophet['ds'].apply(is_autumn)
test_prophet['is_winter'] = test_prophet['ds'].apply(is_winter)
test_prophet['is_weekend'] = test_prophet['ds'].apply(is_weekend)
test_prophet['is_weekday'] = ~test_prophet['ds'].apply(is_weekend)

In [ ]:
# instantiating the class with custom settings
prophet = Prophet(
     daily_seasonality=False,
    weekly_seasonality=False,
    yearly_seasonality=False
)

# custom seasonalities to account for conditional variance 
# (more extreme trends in extreme seasons)
prophet.add_seasonality(name='yearly', period=365.25, fourier_order=10)
prophet.add_seasonality(name='weekly_spring', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_spring')
prophet.add_seasonality(name='weekly_summer', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_summer')
prophet.add_seasonality(name='weekly_autumn', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_autumn')
prophet.add_seasonality(name='weekly_winter', 
                        period=7,
                        fourier_order=5, 
                        condition_name='is_winter')
prophet.add_seasonality(name='daily_spring',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_spring')
prophet.add_seasonality(name='daily_summer',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_summer')
prophet.add_seasonality(name='daily_autumn',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_autumn')
prophet.add_seasonality(name='daily_winter',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_winter')
prophet.add_seasonality(name='daily_weekend',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_weekend')
prophet.add_seasonality(name='daily_weekday',  
                        period=1,
                        fourier_order=5, 
                        condition_name='is_weekday')

# fitting the model
prophet.fit(train_prophet);

# part of the dataframe on which we want to make predictions
future = test_prophet.drop(['y'], axis=1)

# predicting values
forecast = prophet.predict(future)

# see https://github.com/facebook/prophet/issues/999 for the matplotlib_converts()
pd.plotting.register_matplotlib_converters()

# plotting the seasonality components found
_ = prophet.plot_components(forecast)

In [ ]:
forecast.tail()

In [ ]:
# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_prophet.ds, y=test_prophet.y,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.5))
fig.update_layout(title='Prophet Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for Prophet\'s predictions: {mape(test_prophet.y, forecast.yhat)}')

In [ ]:
# interval length
interval = 24 * 7

# intermediary variables for readability
x_true, y_true = test_prophet.iloc[:interval].ds, test_prophet.iloc[:interval].y
x_pred, y_pred = forecast.iloc[:interval].ds, forecast.iloc[:interval].yhat

# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Prophet Intra-Day Forecast of First {interval} Hours of Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for interval of the first {interval} hours: {mape(y_true, y_pred)}')

In [ ]:
# interval length
interval = -24 * 7

# intermediary variables for readability
x_true, y_true = test_prophet.iloc[:interval].ds, test_prophet.iloc[:interval].y
x_pred, y_pred = forecast.iloc[:interval].ds, forecast.iloc[:interval].yhat

# create figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode='lines',
                         name='Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Prophet Intra-Day Forecast of last {abs(interval)} Hours of Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# quantify accuracy
print(f'MAPE for interval of the first {abs(interval)} hours: {mape(y_true, y_pred)}')

In [ ]:
# setting up a dataframe for each of the last 10 full years present in the data frame
# 2008
id_date = pd.to_datetime('2008-01-01')
year_incr = pd.DateOffset(years=1)
df_2008 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2009
id_date = pd.to_datetime('2009-01-01')
year_incr = pd.DateOffset(years=1)
df_2009 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2010
id_date = pd.to_datetime('2010-01-01')
year_incr = pd.DateOffset(years=1)
df_2010 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2011
id_date = pd.to_datetime('2011-01-01')
year_incr = pd.DateOffset(years=1)
df_2011 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2012
id_date = pd.to_datetime('2012-01-01')
year_incr = pd.DateOffset(years=1)
df_2012 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2013
id_date = pd.to_datetime('2013-01-01')
year_incr = pd.DateOffset(years=1)
df_2013 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2014
id_date = pd.to_datetime('2014-01-01')
year_incr = pd.DateOffset(years=1)
df_2014 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2015
id_date = pd.to_datetime('2015-01-01')
year_incr = pd.DateOffset(years=1)
df_2015 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2016
id_date = pd.to_datetime('2016-01-01')
year_incr = pd.DateOffset(years=1)
df_2016 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()
#2017
id_date = pd.to_datetime('2017-01-01')
year_incr = pd.DateOffset(years=1)
df_2017 = df.loc[(df.index >= id_date) & (df.index < id_date + year_incr) ].copy()

In [ ]:
# preparing the yearly dataframes for predictions and plots
df_2008 = df_2008.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2008['is_spring'] = df_2008['ds'].apply(is_spring)
df_2008['is_summer'] = df_2008['ds'].apply(is_summer)
df_2008['is_autumn'] = df_2008['ds'].apply(is_autumn)
df_2008['is_winter'] = df_2008['ds'].apply(is_winter)
df_2008['is_weekend'] = df_2008['ds'].apply(is_weekend)
df_2008['is_weekday'] = ~df_2008['ds'].apply(is_weekend)

df_2009 = df_2009.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2009['is_spring'] = df_2009['ds'].apply(is_spring)
df_2009['is_summer'] = df_2009['ds'].apply(is_summer)
df_2009['is_autumn'] = df_2009['ds'].apply(is_autumn)
df_2009['is_winter'] = df_2009['ds'].apply(is_winter)
df_2009['is_weekend'] = df_2009['ds'].apply(is_weekend)
df_2009['is_weekday'] = ~df_2009['ds'].apply(is_weekend)

df_2010 = df_2010.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2010['is_spring'] = df_2010['ds'].apply(is_spring)
df_2010['is_summer'] = df_2010['ds'].apply(is_summer)
df_2010['is_autumn'] = df_2010['ds'].apply(is_autumn)
df_2010['is_winter'] = df_2010['ds'].apply(is_winter)
df_2010['is_weekend'] = df_2010['ds'].apply(is_weekend)
df_2010['is_weekday'] = ~df_2010['ds'].apply(is_weekend)

df_2011 = df_2011.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2011['is_spring'] = df_2011['ds'].apply(is_spring)
df_2011['is_summer'] = df_2011['ds'].apply(is_summer)
df_2011['is_autumn'] = df_2011['ds'].apply(is_autumn)
df_2011['is_winter'] = df_2011['ds'].apply(is_winter)
df_2011['is_weekend'] = df_2011['ds'].apply(is_weekend)
df_2011['is_weekday'] = ~df_2011['ds'].apply(is_weekend)

df_2012 = df_2012.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2012['is_spring'] = df_2012['ds'].apply(is_spring)
df_2012['is_summer'] = df_2012['ds'].apply(is_summer)
df_2012['is_autumn'] = df_2012['ds'].apply(is_autumn)
df_2012['is_winter'] = df_2012['ds'].apply(is_winter)
df_2012['is_weekend'] = df_2012['ds'].apply(is_weekend)
df_2012['is_weekday'] = ~df_2012['ds'].apply(is_weekend)

df_2013 = df_2013.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2013['is_spring'] = df_2013['ds'].apply(is_spring)
df_2013['is_summer'] = df_2013['ds'].apply(is_summer)
df_2013['is_autumn'] = df_2013['ds'].apply(is_autumn)
df_2013['is_winter'] = df_2013['ds'].apply(is_winter)
df_2013['is_weekend'] = df_2013['ds'].apply(is_weekend)
df_2013['is_weekday'] = ~df_2013['ds'].apply(is_weekend)

df_2014 = df_2014.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2014['is_spring'] = df_2014['ds'].apply(is_spring)
df_2014['is_summer'] = df_2014['ds'].apply(is_summer)
df_2014['is_autumn'] = df_2014['ds'].apply(is_autumn)
df_2014['is_winter'] = df_2014['ds'].apply(is_winter)
df_2014['is_weekend'] = df_2014['ds'].apply(is_weekend)
df_2014['is_weekday'] = ~df_2014['ds'].apply(is_weekend)

df_2015 = df_2015.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2015['is_spring'] = df_2015['ds'].apply(is_spring)
df_2015['is_summer'] = df_2015['ds'].apply(is_summer)
df_2015['is_autumn'] = df_2015['ds'].apply(is_autumn)
df_2015['is_winter'] = df_2015['ds'].apply(is_winter)
df_2015['is_weekend'] = df_2015['ds'].apply(is_weekend)
df_2015['is_weekday'] = ~df_2015['ds'].apply(is_weekend)

df_2016 = df_2016.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2016['is_spring'] = df_2016['ds'].apply(is_spring)
df_2016['is_summer'] = df_2016['ds'].apply(is_summer)
df_2016['is_autumn'] = df_2016['ds'].apply(is_autumn)
df_2016['is_winter'] = df_2016['ds'].apply(is_winter)
df_2016['is_weekend'] = df_2016['ds'].apply(is_weekend)
df_2016['is_weekday'] = ~df_2016['ds'].apply(is_weekend)

df_2017 = df_2017.rename(columns={
                        'date_and_time':'ds',
                        'EnergyDemand_in_MW':'y'
                    })
df_2017['is_spring'] = df_2017['ds'].apply(is_spring)
df_2017['is_summer'] = df_2017['ds'].apply(is_summer)
df_2017['is_autumn'] = df_2017['ds'].apply(is_autumn)
df_2017['is_winter'] = df_2017['ds'].apply(is_winter)
df_2017['is_weekend'] = df_2017['ds'].apply(is_weekend)
df_2017['is_weekday'] = ~df_2017['ds'].apply(is_weekend)

In [ ]:
# forecasting y in each yearly dataframe using the fitted model
df_forecast_2008 = prophet.predict(df_2008)
df_forecast_2009 = prophet.predict(df_2009)
df_forecast_2010 = prophet.predict(df_2010)
df_forecast_2011 = prophet.predict(df_2011)
df_forecast_2012 = prophet.predict(df_2012)
df_forecast_2013 = prophet.predict(df_2013)
df_forecast_2014 = prophet.predict(df_2014)
df_forecast_2015 = prophet.predict(df_2015)
df_forecast_2016 = prophet.predict(df_2016)
df_forecast_2017 = prophet.predict(df_2017)

In [ ]:
# Dropping the year from the ds column for each yearly dataframe
df_2008['ds'] = df_2008['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2009['ds'] = df_2009['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2010['ds'] = df_2010['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2011['ds'] = df_2011['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2012['ds'] = df_2012['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2013['ds'] = df_2013['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2014['ds'] = df_2014['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2015['ds'] = df_2015['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2016['ds'] = df_2016['ds'].dt.strftime('%m-%d %H:%M:%S')
df_2017['ds'] = df_2017['ds'].dt.strftime('%m-%d %H:%M:%S')

In [ ]:
df_2010.head()

In [ ]:
df_with_year = df

In [ ]:
# Dropping the year from the ds column for each yearly forecast dataframe to prepare it for the GUI
df_forecast_2008['ds'] = df_forecast_2008['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2009['ds'] = df_forecast_2009['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2010['ds'] = df_forecast_2010['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2011['ds'] = df_forecast_2011['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2012['ds'] = df_forecast_2012['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2013['ds'] = df_forecast_2013['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2014['ds'] = df_forecast_2014['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2015['ds'] = df_forecast_2015['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2016['ds'] = df_forecast_2016['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2017['ds'] = df_forecast_2017['ds'].dt.strftime('%m-%d %H:%M:%S')

In [ ]:
df_forecast_2010.tail()

In [ ]:
# Generating the dataframe for the five year forecast for the GUI starting from 2020
future_data = prophet.make_future_dataframe(periods=77490, freq='H')

In [ ]:
future_data['is_spring'] = future_data['ds'].apply(is_spring)
future_data['is_summer'] = future_data['ds'].apply(is_summer)
future_data['is_autumn'] = future_data['ds'].apply(is_autumn)
future_data['is_winter'] = future_data['ds'].apply(is_winter)
future_data['is_weekend'] = future_data['ds'].apply(is_weekend)
future_data['is_weekday'] = ~future_data['ds'].apply(is_weekend)

In [ ]:
forecast_data = prophet.predict(future_data)

In [ ]:
forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
prophet.plot(forecast_data);

In [ ]:
# Generating the data frames for use in the GUI
id_date_2020 = pd.to_datetime('2020-01-01 00:00:00')
df_forecast_2020 = forecast_data.loc[(forecast_data.ds >= id_date_2020) & (forecast_data.ds < id_date_2020+year_incr) ].copy()
id_date_2021 = pd.to_datetime('2021-01-01 00:00:00')
df_forecast_2021 = forecast_data.loc[(forecast_data.ds >= id_date_2021) & (forecast_data.ds < id_date_2021+year_incr) ].copy()
id_date_2022 = pd.to_datetime('2022-01-01 00:00:00')
df_forecast_2022 = forecast_data.loc[(forecast_data.ds >= id_date_2022) & (forecast_data.ds < id_date_2022+year_incr) ].copy()
id_date_2023 = pd.to_datetime('2023-01-01 00:00:00')
df_forecast_2023 = forecast_data.loc[(forecast_data.ds >= id_date_2023) & (forecast_data.ds < id_date_2023+year_incr) ].copy()
id_date_2024 = pd.to_datetime('2024-01-01 00:00:00')
df_forecast_2024 = forecast_data.loc[(forecast_data.ds >= id_date_2024) & (forecast_data.ds < id_date_2024+year_incr) ].copy()

In [ ]:
df_forecast_2020.tail()

In [ ]:
# Preparing yearly forecast data frames for the use in GUI
df_forecast_2020['ds'] = df_forecast_2020['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2021['ds'] = df_forecast_2021['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2022['ds'] = df_forecast_2022['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2023['ds'] = df_forecast_2023['ds'].dt.strftime('%m-%d %H:%M:%S')
df_forecast_2024['ds'] = df_forecast_2024['ds'].dt.strftime('%m-%d %H:%M:%S')

In [ ]:
df_forecast_2020.tail()

In [ ]:
#Saving data frames
# Yearly actual data
df_with_year.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_with_year.csv', index = False)
df_2008.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2008.csv', index = False)
df_2009.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2009.csv', index = False)
df_2010.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2010.csv', index = False)
df_2011.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2011.csv', index = False)
df_2012.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2012.csv', index = False)
df_2013.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2013.csv', index = False)
df_2014.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2014.csv', index = False)
df_2015.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2015.csv', index = False)
df_2016.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2016.csv', index = False)
df_2017.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_2017.csv', index = False)
#Yearly forecasted data
df_forecast_2008.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2008.csv', index = False)
df_forecast_2009.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2009.csv', index = False)
df_forecast_2010.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2010.csv', index = False)
df_forecast_2011.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2011.csv', index = False)
df_forecast_2012.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2012.csv', index = False)
df_forecast_2013.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2013.csv', index = False)
df_forecast_2014.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2014.csv', index = False)
df_forecast_2015.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2015.csv', index = False)
df_forecast_2016.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2016.csv', index = False)
df_forecast_2017.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2017.csv', index = False)
df_forecast_2020.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2020.csv', index = False)
df_forecast_2021.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2021.csv', index = False)
df_forecast_2022.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2022.csv', index = False)
df_forecast_2023.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2023.csv', index = False)
df_forecast_2024.to_csv('/home/jovyan/Capstone_Data_Files/Capstone_DataFrames/df_forecast_2024.csv', index = False)